In [2]:
import pandas as pd
import re
import warnings; warnings.simplefilter('ignore')
import numpy as np
import warnings
import os
import sys
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.data.read_parallel import read_parallel_local

In [3]:
df = pd.read_csv("temp_vn.csv", encoding="latin1", parse_dates=True)
print(len(df))
df = df[~df.id.isna()]
df.id = df.id.astype(int)

# Sample a random version of each bill, at the end we only want
# one version of the bill in the dataset.
df_one_bill = df[['bill_id', 'version_number']].groupby('bill_id').sample(n=1, random_state=0)
df = df_one_bill.merge(df[['id', 'version_number', 'bill_id', 'partisan_lean', 'sc_id', 'signed']], on = ['version_number', 'bill_id'])

print(len(df))
df = df.sample(int(len(df)/1.3))  # Max that fits in local memory right now
print(len(df))


df['text'] = read_parallel_local(df['id'], "../data/clean/")
df = df.reset_index(drop=True)

485388
199451
153423
Took 6.076287909348806 min to open 153423 files with 20 processes.


In [4]:
df.head()

,bill_id,version_number,id,partisan_lean,sc_id,signed,text
0,1227085,1,2375742,0.754207,580-2,0,x back button image: click to go to previous ...
1,1340732,1,2682190,0.236118,562-1,1,<billno> <sponsor> senate joint resolution by...
2,1263736,2,2519963,0.272146,628-1,1,*sb0177.1* january 2020 senate bill no. diges...
3,1200331,3,2659359,0.396835,563-2,0,by:aalang h.b.ano.a3851 a bill to be entitled...
4,1097029,1,2126414,0.127800,576-1,1,| | | search results close toggle navigation ...


In [5]:
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
         
data_words = list(sent_to_words(df.text.values.tolist()))

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Filter out extremely rare words and extremely common
id2word.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [6]:
print(data_words[0])
df.text[0]

['back', 'button', 'image', 'click', 'to', 'go', 'to', 'previous', 'page', 'quick', 'search', 'bill', 'numberbill', 'keyword', 'text', 'bill', 'text', 'bill', 'information', 'hr', 'share', 'this', 'bill', 'start', 'california', 'legislature', 'regular', 'session', 'house', 'resolution', 'no', 'introduced', 'by', 'assembly', 'member', 'kamlager', 'dove', 'may', 'relative', 'to', 'the', 'university', 'of', 'california', 'los', 'angeles', 'legislative', 'counsel', 'digest', 'hr', 'as', 'introduced', 'kamlager', 'dove', 'digest', 'key', 'bill', 'text', 'whereas', 'the', 'university', 'of', 'california', 'los', 'angeles', 'was', 'founded', 'on', 'may', 'and', 'whereas', 'in', 'classes', 'began', 'at', 'its', 'current', 'campus', 'in', 'the', 'westwood', 'neighborhood', 'of', 'los', 'angeles', 'and', 'whereas', 'ucla', 'consistently', 'receives', 'more', 'applications', 'for', 'undergraduate', 'admission', 'than', 'any', 'other', 'university', 'in', 'the', 'country', 'with', 'over', 'applica

" x back button image: click to go to previous page   quick search: bill numberbill keyword  >> >> text  bill text bill information hr-37            share this: bill start california legislature regular session house resolution no. 37-introduced by assembly member kamlager-dove  may 2019 relative to the university of california, los angeles. legislative counsel's digest hr as introduced, kamlager-dove. digest key bill text whereas, the university of california, los angeles, was founded on may 1919; and whereas, in classes began at its current campus in the westwood neighborhood of los angeles; and whereas, ucla consistently receives more applications for undergraduate admission than any other university in the country, with over applications received for the academic year; and whereas, in u.s. news and world reports overall hospital rankings, the ronald reagan ucla medical center is ranked first in los angeles, second in california, and seventh in the country; and whereas, ucla is the 

In [7]:
n_topics = 40
n_passes = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=n_topics,
                                       passes=n_passes)

NameError: name 'pprint' is not defined

In [8]:
doc_lda = lda_model[corpus]

In [9]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./lda_results/ldavis_prepared_'+str(num_topics))

#try:
#    ! mkdir lda_results
    
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './lda_results/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
22     0.153487 -0.016712       1        1  5.245854
27    -0.085833  0.169266       2        1  4.886996
29    -0.038228  0.087561       3        1  4.445430
25     0.036626 -0.020186       4        1  4.009046
30     0.129016  0.014114       5        1  3.940973
31     0.056870  0.143990       6        1  3.887261
11    -0.153301  0.060937       7        1  3.537951
37    -0.123293 -0.039008       8        1  3.447891
0     -0.071631  0.032658       9        1  3.329005
36    -0.145449  0.160201      10        1  3.263848
18     0.111569  0.002643      11        1  3.101939
7     -0.021668 -0.113346      12        1  3.055105
8     -0.009594 -0.079586      13        1  2.965274
12     0.123197  0.040206      14        1  2.897061
3      0.149350  0.012737      15        1  2.875975
5      0.045347  0.050687      16        1  2.755904
20    -0.008540 -0.013921      17        1  2.683605
19    -0.049195 -0.171314      18        1  2.547600
32     0.110725 -0.072842      19        1  2.523840
13     0.111395  0.092996      20        1  2.427677
23     0.090142  0.027414      21        1  2.383420
26    -0.033893  0.202045      22        1  2.343736
1      0.048374 -0.023974      23        1  2.260387
39    -0.193665  0.145725      24        1  2.225482
6      0.058272  0.075530      25        1  2.154743
28     0.150355  0.017395      26        1  2.048274
4      0.146409 -0.098797      27        1  1.811672
15    -0.097291  0.004492      28        1  1.737267
16     0.116100  0.009678      29        1  1.707433
38     0.106852  0.029593      30        1  1.648137
24     0.005570 -0.135738      31        1  1.610212
9     -0.052281 -0.009958      32        1  1.507189
14     0.125439  0.014982      33        1  1.464049
33    -0.053522 -0.123552      34        1  1.389924
21    -0.028193 -0.108917      35        1  1.152846
34    -0.100814 -0.156661      36        1  1.034184
17    -0.272724 -0.016491      37        1  1.011024
2     -0.081511  0.032037      38        1  0.983622
35    -0.218584 -0.098952      39        1  0.866979
10    -0.035888 -0.126933      40        1  0.831183, topic_info=           Term           Freq          Total Category  logprob  loglift
2013       line  638230.000000  638230.000000  Default  30.0000  30.0000
265      school  460720.000000  460720.000000  Default  29.0000  29.0000
327        code  410488.000000  410488.000000  Default  28.0000  28.0000
1449       fund  352207.000000  352207.000000  Default  27.0000  27.0000
151         tax  334648.000000  334648.000000  Default  26.0000  26.0000
...         ...            ...            ...      ...      ...      ...
349   effective    7055.594450  145870.343138  Topic40  -4.9813   1.7614
227         her    6172.765907  120866.504574  Topic40  -5.1150   1.8158
933         his    5864.149149  177559.829788  Topic40  -5.1663   1.3799
296         who    5054.845022  458757.369030  Topic40  -5.3148   0.2821
452     article    4938.179703  192382.609479  Topic40  -5.3382   1.1278

[3510 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
4367       1  0.000106      aa
4367       3  0.014521      aa
4367       5  0.001166      aa
4367       6  0.003074      aa
4367       8  0.007526      aa
...      ...       ...     ...
4504      29  0.023600  zoning
4504      30  0.012501  zoning
4504      32  0.039290  zoning
4504      33  0.002041  zoning
34003     17  0.985458   zuber

[33347 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[23, 28, 30, 26, 31, 32, 12, 38, 1, 37, 19, 8, 9, 13, 4, 6, 21, 20, 33, 14, 24, 27, 2, 40, 7, 29, 5, 16, 17, 39, 25, 10, 15, 34, 22, 35, 18, 3, 36, 11])

In [ ]:
def dense_vector(ldav, n):
    v = [0]*n
    for (i, p) in ldav:
        v[i] = p
    return tuple(v)

doc_topics = [dense_vector(lda_model[x], num_topics) for x in corpus]
features = np.asarray(doc_topics)
print(f"Features shape:{features.shape}.")


In [19]:
n_topics = 40
n_passes = 10
# n_topics, n_passes
file_name = "bow_models/lda_" + str(n_topics) + "_" + str(n_passes) + ".dat"
if not os.path.isdir("bow_models"):
    os.mkdir("bow_models")
    
with open(file_name, "wb") as f:
    pickle.dump([lda_model, id2word], f)



/home/luke/repos/govhawk_ml/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
from gensim.models import LsiModel
lsi_model = LsiModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=n_topics)

file_name = "bow_models/lsi_" + str(n_topics) + ".dat"
with open(file_name, "wb") as f:
    pickle.dump([lsi_model, id2word], f)

/home/luke/repos/govhawk_ml/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
